<a href="https://colab.research.google.com/github/wasilone11/grroom/blob/main/Copy_of_tiny%2B_merged_model_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from IPython.display import HTML
from subprocess import getoutput
s = getoutput('nvidia-smi')
print(s)
if 'K80' in s:
    gpu = 'K80'
elif 'T4' in s:
    gpu = 'T4'
elif 'P100' in s:
    gpu = 'P100'
else:
    gpu='DONT PROCEED'
display(HTML(f"<h1>{gpu}</h1>"))

Mon Apr 26 08:13:52 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.19.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   52C    P8    11W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [3]:
%cd /gdrive/MyDrive/darknet
!chmod +x ./darknet
!./darknet

/gdrive/MyDrive/darknet
usage: ./darknet <function>


In [4]:
import os
import random
folder="footwear_merged"
category="f_heel_width_4e_case_1_2"
if category=="":
    inputpath="../"+folder
    opPath=folder+"_tiny_model+"
else:
    inputpath="../"+folder+'/'+category
    opPath=folder+"_"+category+"_tiny_model+"
if opPath not in os.listdir():
    !mkdir $opPath

In [5]:
d={}
for i in [m for m in os.listdir(inputpath) if "." not in m]:
 
    all_temp=[]
    for j in [m for m in os.listdir(inputpath+"/"+i) if "." not in m]:
        temp=[]
        for k in [m for m in os.listdir(inputpath+"/"+i+"/"+j) if "." not in m]:
            tmp_lst=[inputpath+'/'+i+'/'+j+'/'+k+'/'+m for m in os.listdir(inputpath+'/'+i+'/'+j+'/'+k)]
            temp.extend(tmp_lst)
 
        temp=[m for m in temp if (m[:3]!="ls:")&("classes" not in m)]
        temp.sort()
        images=[m for m in temp if ".txt" not in m]
        txts=[m for m in temp if ".txt" in m]
        all_temp.extend(temp)
        print(folder,i,j,len(images),len(txts))
    d[i]=all_temp

footwear_merged no_heel ballerinas 474 474
footwear_merged no_heel boots 338 338
footwear_merged no_heel classic_shoe 473 473
footwear_merged no_heel sandal_shoe 318 318
footwear_merged no_heel slippers 490 490
footwear_merged no_heel sneakers 310 310
footwear_merged slim_heel ballerinas 530 530
footwear_merged slim_heel boots 547 547
footwear_merged slim_heel pumps 536 536
footwear_merged slim_heel sandal_shoe 533 533
footwear_merged slim_heel slippers 555 555
footwear_merged wide_heel ballerinas 485 485
footwear_merged wide_heel boots 488 488
footwear_merged wide_heel classic_shoe 488 488
footwear_merged wide_heel pumps 305 305
footwear_merged wide_heel sandal_shoe 463 463
footwear_merged wide_heel slippers 415 415
footwear_merged wide_heel sneakers 467 467


In [6]:
tmp=list(d.keys())
for i in tmp:
    print(i,len(d[i]))

no_heel 4806
slim_heel 5402
wide_heel 6222


In [7]:
train=[]
for i in d.keys():
    for j in range(len(d[i])-1):
        if d[i][j][:d[i][j].rfind('.')]==d[i][j+1][:d[i][j+1].rfind('.')]:
            train.append(d[i][j])
            j+=1

In [8]:
count=0
classes={}
for i in d.keys():
    classes[count]=i
    for j in d[i]:
        if ".txt" in j:
            f=open(j,'r+')
            s=[k.split(" ") for k in f.read().split('\n')[:-1]]
            s=[[str(count)]+k[1:] for k in s]+[['']]
            s="\n".join([" ".join(k) for k in s])
            f.seek(0)
            f.write(s)
            f.close()
    count+=1

In [9]:
f=open(inputpath+"/classes.txt",'w')
c=[]
for i in range(len(classes)):
    c.append(classes[i])
f.write("\n".join(c))
f.close()

In [10]:
!cat $inputpath/classes.txt

no_heel
slim_heel
wide_heel

In [11]:
classesPath=inputpath+"/classes.txt"
f=open(classesPath,"r")
names=f.read()
f.close()
f=open(inputpath+"/obj.names","w")
f.write(names)
f.close()

In [12]:
numClass=len(names.split("\n"))
objdata=["classes= "+str(numClass),
         "train = "+inputpath+"/train.txt",
         "valid = "+inputpath+"/valid.txt",
         "names = "+inputpath+"/obj.names",
         "backup = "+opPath+"/"]
f=open(inputpath+"/obj.data","w")
f.write("\n".join(objdata))
f.close()

In [13]:
train=[i for i in train if ".txt" not in i]
train=[i for i in train if ".gif" not in i]
train=[i for i in train if ".jfif" not in i]
random.shuffle(train)
l=len(train)
valid=train[int(l*5/6):]
train=train[:int(l*5/6)]
f=open(inputpath+'/train.txt','w')
f.write("\n".join(train))
f.close()
f=open(inputpath+'/valid.txt','w')
f.write("\n".join(valid))
f.close()

In [14]:
numClass

3

In [15]:
if "yolov4-tiny-custom.cfg" not in os.listdir(inputpath):
    !cp cfg/yolov4-tiny-custom.cfg $inputpath/yolov4-tiny-custom.cfg
else:
    !rm $inputpath/yolov4-tiny-custom.cfg
    !cp cfg/yolov4-tiny-custom.cfg $inputpath/yolov4-tiny-custom.cfg

In [16]:
46#change number of class and filters
comm='s/classes=80/classes='+str(numClass)+"/"
!sed -i $comm $inputpath/yolov4-tiny-custom.cfg
comm='s/filters=255/filters='+str((numClass+5)*3)+"/"
!sed -i $comm $inputpath/yolov4-tiny-custom.cfg

#change number of steps and max batches
max_batches=max(6000,(numClass)*2000)
comm='s/max_batches\ =\ 500200/max_batches\ =\ '+str(max_batches)+"/"
!sed -i $comm $inputpath/yolov4-tiny-custom.cfg
comm='s/steps=400000,450000/steps='+str(int(0.8*max_batches))+','+str(int(0.9*max_batches))+"/"
!sed -i $comm $inputpath/yolov4-tiny-custom.cfg

netSize=416  #can be multiple of 32 or 608(but makes training slow)
comm='s/width=608/width='+str(netSize)+"/"
!sed -i $comm $inputpath/yolov4-tiny-custom.cfg
comm='s/height=608/height='+str(netSize)+"/"
!sed -i $comm $inputpath/yolov4-tiny-custom.cfg

batch=64
subdiv=2
comm='s/batch=64/batch='+str(batch)+'/'
!sed -i $comm $inputpath/yolov4-tiny-custom.cfg
comm='s/subdivisions=16/subdivisions='+str(subdiv)+'/'
!sed -i $comm $inputpath/yolov4-tiny-custom.cfg

In [17]:
if "yolov4.conv.137" not in os.listdir():
    !wget https://github.com/AlexeyAB/darknet/releases/download/darknet_yolo_v3_optimal/yolov4.conv.137
if "yolov4-tiny.weights" not in os.listdir():
    !wget https://github.com/AlexeyAB/darknet/releases/download/darknet_yolo_v4_pre/yolov4-tiny.weights
if "yolov4-tiny.conv.29" not in os.listdir():
    !wget https://github.com/AlexeyAB/darknet/releases/download/darknet_yolo_v4_pre/yolov4-tiny.conv.29

In [18]:
!./darknet detector train $inputpath/obj.data $inputpath/yolov4-tiny-custom.cfg yolov4-tiny.conv.29 -dont_show -map

Streaming output truncated to the last 5000 lines.

 Tensor Cores are used.
 Last accuracy mAP@0.5 = 69.47 %, best = 69.47 % 
 5506: 0.374191, 0.392541 avg loss, 0.000026 rate, 0.468725 seconds, 352384 images, 0.197378 hours left
Loaded: 0.733090 seconds - performance bottleneck on CPU or Disk HDD/SSD
v3 (iou loss, Normalizer: (iou: 0.07, obj: 1.00, cls: 1.00) Region 30 Avg (IOU: 0.759346), count: 34, class_loss = 0.322928, iou_loss = 0.302942, total_loss = 0.625870 
v3 (iou loss, Normalizer: (iou: 0.07, obj: 1.00, cls: 1.00) Region 37 Avg (IOU: 0.717575), count: 48, class_loss = 0.454353, iou_loss = 3.383499, total_loss = 3.837852 
 total_bbox = 451627, rewritten_bbox = 0.037642 % 

 (next mAP calculation at 5697 iterations) 

 Tensor Cores are used.
 Last accuracy mAP@0.5 = 69.47 %, best = 69.47 % 
 5507: 0.388927, 0.392179 avg loss, 0.000026 rate, 0.509550 seconds, 352448 images, 0.197247 hours left
Loaded: 0.607447 seconds - performance bottleneck on CPU or Disk HDD/SSD
v3 (iou los

In [19]:
!./darknet detector train $inputpath/obj.data $inputpath/yolov4-tiny-custom.cfg $opPath/yolov4-tiny-custom_last.weights -dont_show -map

 CUDA-version: 11000 (11020), cuDNN: 7.6.5, CUDNN_HALF=1, GPU count: 1  
 CUDNN_HALF=1 
 OpenCV version: 3.2.0
 Prepare additional network for mAP calculation...
 0 : compute_capability = 750, cudnn_half = 1, GPU: Tesla T4 
net.optimized_memory = 0 
mini_batch = 1, batch = 1, time_steps = 1, train = 0 
   layer   filters  size/strd(dil)      input                output
   0 conv     32       3 x 3/ 2    416 x 416 x   3 ->  208 x 208 x  32 0.075 BF
   1 conv     64       3 x 3/ 2    208 x 208 x  32 ->  104 x 104 x  64 0.399 BF
   2 conv     64       3 x 3/ 1    104 x 104 x  64 ->  104 x 104 x  64 0.797 BF
   3 route  2 		                       1/2 ->  104 x 104 x  32 
   4 conv     32       3 x 3/ 1    104 x 104 x  32 ->  104 x 104 x  32 0.199 BF
   5 conv     32       3 x 3/ 1    104 x 104 x  32 ->  104 x 104 x  32 0.199 BF
   6 route  5 4 	                           ->  104 x 104 x  64 
   7 conv     64       1 x 1/ 1    104 x 104 x  64 ->  104 x 104 x  64 0.089 BF
   8 route  2 7 	  

In [ ]:
!./darknet detector map $inputpath/obj.data $inputpath/yolov4-tiny-custom.cfg $opPath/yolov4-tiny-custom_best.weights

 CUDA-version: 11000 (11020), cuDNN: 7.6.5, CUDNN_HALF=1, GPU count: 1  
 CUDNN_HALF=1 
 OpenCV version: 3.2.0
 0 : compute_capability = 750, cudnn_half = 1, GPU: Tesla T4 
net.optimized_memory = 0 
mini_batch = 1, batch = 1, time_steps = 1, train = 0 
   layer   filters  size/strd(dil)      input                output
   0 conv     32       3 x 3/ 2    416 x 416 x   3 ->  208 x 208 x  32 0.075 BF
   1 conv     64       3 x 3/ 2    208 x 208 x  32 ->  104 x 104 x  64 0.399 BF
   2 conv     64       3 x 3/ 1    104 x 104 x  64 ->  104 x 104 x  64 0.797 BF
   3 route  2 		                       1/2 ->  104 x 104 x  32 
   4 conv     32       3 x 3/ 1    104 x 104 x  32 ->  104 x 104 x  32 0.199 BF
   5 conv     32       3 x 3/ 1    104 x 104 x  32 ->  104 x 104 x  32 0.199 BF
   6 route  5 4 	                           ->  104 x 104 x  64 
   7 conv     64       1 x 1/ 1    104 x 104 x  64 ->  104 x 104 x  64 0.089 BF
   8 route  2 7 	                           ->  104 x 104 x 128 
   9 